In [1]:
import numpy as np
import pandas as pd
import datetime
from tqdm import tqdm 

import warnings 
warnings.filterwarnings('ignore')

In [2]:
#读取数据
my_data=pd.read_csv(
    'IF8888.CCFX_5m.csv', sep=',', parse_dates=True)
my_data = my_data[my_data["date"]<='2012-08-24']
my_data.head()

,date,time,dates,open,close,high,low,volume,money
0,2010-04-16,09:20:00,2010/4/16 9:20,3474.055,3476.633,3513.114,3471.577,2243,2.348769e+09
1,2010-04-16,09:25:00,2010/4/16 9:25,3476.431,3478.663,3483.076,3474.024,1209,1.263114e+09
2,2010-04-16,09:30:00,2010/4/16 9:30,3478.663,3466.988,3479.438,3458.142,1844,1.919536e+09
3,2010-04-16,09:35:00,2010/4/16 9:35,3467.864,3472.736,3478.529,3463.725,1731,1.802158e+09
4,2010-04-16,09:40:00,2010/4/16 9:40,3472.445,3467.961,3475.047,3465.260,1337,1.389845e+09


In [3]:
day_first_idexs = my_data.drop_duplicates('date').index.values #取出相同date的第一个日期index
day_idexs = np.hstack([day_first_idexs,my_data.shape[0]]) #加上最后一个index（方便后面取每日区间）
my_data["rank"]=None
rank = np.arange(1,55)
# print(rank)
for day_begin, day_end in zip(tqdm(day_idexs[:-1]), day_idexs[1:]):#取每一天的区间        
        my_data.loc[my_data.iloc[day_begin: day_end, :].index.tolist(),"rank"] = rank
#         print(my_data.iloc[day_begin: day_end, :])
        
my_data       

100%|██████████████████████████████████████████████████████████████████████████████| 575/575 [00:00<00:00, 1298.51it/s]


,date,time,dates,open,close,high,low,volume,money,rank
0,2010-04-16,09:20:00,2010/4/16 9:20,3474.055,3476.633,3513.114,3471.577,2243,2.348769e+09,1
1,2010-04-16,09:25:00,2010/4/16 9:25,3476.431,3478.663,3483.076,3474.024,1209,1.263114e+09,2
2,2010-04-16,09:30:00,2010/4/16 9:30,3478.663,3466.988,3479.438,3458.142,1844,1.919536e+09,3
3,2010-04-16,09:35:00,2010/4/16 9:35,3467.864,3472.736,3478.529,3463.725,1731,1.802158e+09,4
4,2010-04-16,09:40:00,2010/4/16 9:40,3472.445,3467.961,3475.047,3465.260,1337,1.389845e+09,5
...,...,...,...,...,...,...,...,...,...,...
31045,2012-08-24,14:55:00,2012/8/24 14:55,2301.385,2301.393,2302.336,2301.035,4184,2.883473e+09,50
31046,2012-08-24,15:00:00,2012/8/24 15:00,2301.207,2300.002,2301.554,2299.396,4533,3.122195e+09,51
31047,2012-08-24,15:05:00,2012/8/24 15:05,2299.996,2300.188,2300.695,2299.996,2266,1.560776e+09,52
31048,2012-08-24,15:10:00,2012/8/24 15:10,2300.180,2299.994,2300.751,2299.816,3091,2.129082e+09,53


# 分割train和test

In [4]:
my_data_train = my_data[my_data['date']<='2011-12-30']
my_data_test = my_data[(my_data['date']>'2011-12-30')]

In [5]:
x_train = my_data_train[['high', 'low', 'close', 'volume', 'rank']].values
# print(x_train[:,4])
y_train = (my_data_train['close']/my_data_train['close'].shift(1)-1).fillna(0).values

x_test = my_data_test[['high', 'low', 'close', 'volume', 'rank']].values
y_test = (my_data_test['close']/my_data_test['close'].shift(1)-1).fillna(0).values

## 遗传规划

In [6]:
import geppy as gep
from deap import creator, base, tools
import numpy as np
import random
import math
from sympy import *


def protected_div(x1, x2):
    if abs(x2) < 1e-6:
        return 1
    return x1 / x2

def protected_acos(x1):
    if x1<-1 or x1>1:
        return 1
    return math.acos(x1)

def protected_pow(x1,x2):
    if int(x2)==x2 and -1<=x1<=1 and 0<=x2 and x2<=100:
        return math.pow(x1,x2)
    return 1

import operator 

pset = gep.PrimitiveSet('Main', input_names=['x1','x2','x3', 'x4','x5'])
pset.add_function(operator.add, 2)
pset.add_function(operator.sub, 2)
pset.add_function(operator.mul, 2)
pset.add_function(protected_div, 2)
pset.add_function(math.sin, 1)
pset.add_function(math.cos, 1)
pset.add_function(math.tan, 1)
pset.add_function(protected_pow, 2)
pset.add_function(protected_acos, 1)
# pset.add_function(math.atan, 1)
pset.add_ephemeral_terminal(name='enc', gen=lambda: random.randint(-5, 5)) # each ENC is a random integer within [-10, 10]

from deap import creator, base, tools

creator.create("Fitness", base.Fitness, weights=(1,))  # to maximize the objective (fitness)
creator.create("Individual", gep.Chromosome, fitness=creator.Fitness)

h = 10 # head length
n_genes = 2   # number of genes in a chromosome

toolbox = gep.Toolbox()
toolbox.register('gene_gen', gep.Gene, pset=pset, head_length=h)
toolbox.register('individual', creator.Individual, gene_gen=toolbox.gene_gen, n_genes=n_genes, linker=operator.add)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# compile utility: which translates an individual into an executable function (Lambda)
toolbox.register('compile', gep.compile_, pset=pset)

def evaluate(individual):
    """Evalute the fitness of an individual: 策略累计收益率/回撤越大越好 """
    func = toolbox.compile(individual)
    Yp = np.sign(np.array(list(map(func,  x_train[:, 0], x_train[:, 1],x_train[:, 2],x_train[:, 3],x_train[:, 4]))))
#     Yp = pd.Series(Yp)
#     print(Yp)
    my_data_train['gen_signal'] = Yp
    my_signal_train = calc_signal(my_data_train, strategy=2)
    my_position_train =calc_position(my_signal_train)
    res_my_data, performance_df,fitnes = statistic_performance(my_position_train, 
                                                   data_period=1, 
                                                   is_consider_open=True,
                                                   point = 0.4,
                                                   is_daily_close=True,
                                                   comm=0.00005)
   
    return np.mean(fitnes),

toolbox.register('evaluate', evaluate)
toolbox.register('select', tools.selTournament, tournsize=3)#锦标赛 3选一
# 1. general operators
toolbox.register('mut_uniform', gep.mutate_uniform, pset=pset, ind_pb=0.05, pb=1)
toolbox.register('mut_invert', gep.invert, pb=0.1)
toolbox.register('mut_is_transpose', gep.is_transpose, pb=0.1) #点插串
toolbox.register('mut_ris_transpose', gep.ris_transpose, pb=0.1) #根插串
toolbox.register('mut_gene_transpose', gep.gene_transpose, pb=0.1)  #选两个点交换
toolbox.register('cx_1p', gep.crossover_one_point, pb=0.4) #单点重组
toolbox.register('cx_2p', gep.crossover_two_point, pb=0.2) #双点重组
toolbox.register('cx_gene', gep.crossover_gene, pb=0.1) #两个染色体随机各选一个位置交换
toolbox.register('mut_ephemeral', gep.mutate_uniform_ephemeral, ind_pb='1p')  # 1p: expected one point mutation in an individual
toolbox.pbs['mut_ephemeral'] = 1  # we can also give the probability via the pbs property

# print(np.max)
# print(np.mean)
stats = tools.Statistics(key=lambda ind: ind.fitness.values[0])

stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

# size of population and number of generations
n_pop = 5
n_gen = 2

pop = toolbox.population(n=n_pop)
hof = tools.HallOfFame(3)   # only record the best three individuals ever found in all generations
# record = stats.compile(pop)
# print(record)

In [11]:
# start evolution
pop, log = gep.gep_simple(pop, toolbox, n_generations=n_gen, n_elites=1,
                          stats=stats, hall_of_fame=hof, verbose=True)

import sympy
best_ind = hof[0]
print(best_ind)
symplified_best = gep.simplify(best_ind)
print('Symplified best individual: ')
print(symplified_best)

gen	nevals	avg      	std	min      	max      
0  	5     	-0.960763	0  	-0.960763	-0.960763
1  	4     	-0.960763	0  	-0.960763	-0.960763
2  	4     	-0.960763	0  	-0.960763	-0.960763
add(
	x4,
	tan(tan(sin(add(x2, x3))))
)
Symplified best individual: 
x4 + tan(tan(sin(x2 + x3)))


## 开仓信号

In [8]:
def calc_signal(data, strategy=2):
    """ 计算信号 """
    data["signal"] = data["gen_signal"]
    data["signal"][data["gen_signal"]==0]=None
    data["gen_signal"][data["time"]<'09:30:00']=None
    data["gen_signal"][data["time"]>'15:00:00']=None
    
    ## Strategy 2: 考虑止损
    if strategy==2:
        """ Strategy 2: 
            循环每天进行计算，计算开仓信号signal，
            找第一次有符号变化时刻，发出信号  
        """  
        signals = []
        data["stop_price"]=None
        data["signal"]=None
        data["stop_or_not"]=False
        day_first_idexs = data.drop_duplicates('date').index.values #取出相同date的第一个日期index 如0 240 480
        day_idexs = np.hstack([day_first_idexs,data.shape[0]]) #加上最后一个index 如0 240 480。。。57840（方便后面取每日区间）
        # print(day_first_idexs_train,day_idexs_train)
        
        for day_begin, day_end in zip(day_idexs[:-1], day_idexs[1:]):#取每一天的区间
#             tmp_df = data.iloc[day_begin: day_end, :]
#             print(tmp_df.index)
#             tmp_df['signal'] = None
#             tmp_df['stop_price'] = None      
            direction = None
            stop_price  = None
            
            for i in range(day_begin,day_end):
                if direction is None :
                    if data["gen_signal"][i]==1:
                        data["signal"][i] = 1
                        direction =1
                    elif data["gen_signal"][i]==-1:
                        data["signal"][i] = -1
                        direction = -1
                elif direction == 1 and stop_price is None: 
                    stop_price = data["open"][i]*(1-0.005)
                    data["stop_price"][i] = stop_price
                elif direction == -1 and stop_price is None : 
                    stop_price = data["open"][i]*(1+0.005)
                    data["stop_price"][i] = stop_price
                    
                elif direction ==1 and stop_price != 0:
                    if stop_price-data["low"][i]>=0:
                        data["signal"][i] = 0
                        direction = None
                        stop_price = None
                        data["stop_or_not"][i]=True
                        
                    data["stop_price"][i] = stop_price
                    
                    
                elif direction ==-1 and stop_price != 0:
                    if data["high"][i] - stop_price >=0:  
                        data["signal"][i] = 0
                        direction = None
                        stop_price = None
                        data["stop_or_not"][i]=True
                        
                    data["stop_price"][i] = stop_price
                    
                
                if data["time"][i]=='15:00:00':
                    data["signal"][i] = 0
                    direction = None
                    stop_price =0
#                     data["stop_or_not"][i]=True             
                     
#             data["stop_price"] = data["stop_price"].fillna(method='ffill')
#             data["signal"] = data["signal"].fillna(method='ffill') 
            
            
            
    return data
            

In [9]:
#持仓计算
def calc_position(data, shift_period=1):
    signal = data['signal']
    position = signal.fillna(method='ffill').shift(shift_period).fillna(0)
    data['position'] = position
    return data

In [10]:
def statistic_performance(data, r0=0.03, data_period=1440, 
                          is_consider_open=True,
                          point = None,
                          is_daily_close = False,
                          comm=None, leverage=None):
    position = data['position']
    close = data['close']
    open_p = data['open']
    
    d_first = data['date'].values[0]
    d_last = data['date'].values[-1]
    d_period = datetime.datetime.strptime(d_last, '%Y-%m-%d') - datetime.datetime.strptime(d_first, '%Y-%m-%d')
    y_period = d_period.days / 365

    """      序列型特征 
        hold_r :      持仓收益
        hold_win :    持仓胜负
        hold_cumu_r : 累计持仓收益
        drawdown :    回撤
        ex_hold_r :   超额收益
    """
    """ 默认配置 
        period_in_price：前一日收盘价
        period_out_price：当日收盘价 
    """
    
    """ 每期的持仓收益hold_r是计算的第一个序列，也是后续统计的核心 
        hold_r = hold_out_price/hold_in_price - 1
    """
    # 1. 初始化 
    hold_in_price =  close.shift(1).copy()
    hold_in_price[close.index[0]] = open_p.values[0]
    hold_out_price = close.copy()
              
    # 2. 考虑持仓信号开盘才能操作
    if is_consider_open:
        # 仓位变动第一期的period_in_price为当日open
        hold_in_price[abs(position - position.shift(1).fillna(0))>0] = open_p
        # 仓位变动前最后一期的period_out_price为第二日open (除了整个序列最后一天不变)
        hold_out_price[abs(position - position.shift(-1).fillna(0))>0] = open_p.shift(-1)
        hold_out_price[hold_out_price.index[-1]] = close.values[-1]
    
    #3.考虑止损+跳空 比较止损价和开盘价
    for i in range(data.index[0],data.index[0]+len(data['close'])):
        if data["position"][i]<0 and data['stop_or_not'][i] == True:
            hold_out_price[i] = max(data['stop_price'][i-1],data['open'][i])
        elif data["position"][i]>0 and data['stop_or_not'][i] == True:
            hold_out_price[i] = min(data['stop_price'][i-1],data['open'][i])    
        
    #4.是否收盘前平仓
    if is_daily_close:
        daily_last_idxes = data.drop_duplicates(subset='date',keep='last').index
        #距离每天最后一个时间点的距离 如15:00:00平仓 15:15:00截止  5m一段 hold_out_price.loc[daily_last_idxes.
        hold_out_price.loc[daily_last_idxes.values-3] = close
        
    #5.考虑指数点
    if point:
        # 仓位变动第一期的period_in_price long加指数点 short减指数点
        hold_in_price[position < position.shift(1).fillna(0)] = hold_in_price - point
        hold_in_price[position > position.shift(1).fillna(0)] = hold_in_price + point       
        # 仓位变动前最后一期的period_out_price long减指数点 short加指数点
        hold_out_price[position < position.shift(-1).fillna(0)] = hold_out_price + point
        hold_out_price[position > position.shift(-1).fillna(0)] = hold_out_price - point
    
    # 6.计算 hold_r 
    hold_r = position * (hold_out_price/hold_in_price-1)

    # 7. 考虑换仓成本 
    if comm:
        chgpos_comm_perc = (1-comm)**abs(position - position.shift(1).fillna(0)) 
        hold_r = chgpos_comm_perc * (1+hold_r) - 1 
    # 8.考虑杠杆 
    if leverage:
        hold_r *= leverage

    hold_win = hold_r>0
    hold_cumu_r = (1+hold_r).cumprod() - 1
#     print(hold_cumu_r)
#     print(type(hold_cumu_r))
    drawdown = (hold_cumu_r.cummax()-hold_cumu_r)/(1+hold_cumu_r).cummax()      
    ex_hold_r= hold_r-r0/(250*1440/data_period)
    
    data['hold_in_price'] = hold_in_price *abs(position)
    data['hold_out_price'] = hold_out_price *abs(position)
    data['hold_r'] = hold_r
    data['hold_win'] = hold_win
    data['hold_cumu_r'] = hold_cumu_r
    data['drawdown'] = drawdown
    data['ex_hold_r'] = ex_hold_r
    
    """       数值型特征 
        v_hold_cumu_r：         累计持仓收益
        v_pos_hold_times：      多仓开仓次数
        v_pos_hold_win_times：  多仓开仓盈利次数
        v_pos_hold_period：     多仓持有周期数
        v_pos_hold_win_period： 多仓持有盈利周期数
        v_neg_hold_times：      空仓开仓次数
        v_neg_hold_win_times：  空仓开仓盈利次数
        v_neg_hold_period：     空仓持有盈利周期数
        v_neg_hold_win_period： 空仓开仓次数
        
        v_hold_period：         持仓周期数（最后一笔未平仓订单也算）
        v_hold_win_period：     持仓盈利周期数（最后一笔未平仓订单也算）
        
        v_max_dd：              最大回撤
        v_annual_std：          年化标准差
        v_annual_ret：          年化收益
        v_sharpe：              年化夏普
    """
    v_hold_cumu_r = hold_cumu_r.tolist()[-1]

    v_pos_hold_times= 0 
    v_pos_hold_win_times = 0
    v_pos_hold_period = 0
    v_pos_hold_win_period = 0
    v_neg_hold_times= 0 
    v_neg_hold_win_times = 0
    v_neg_hold_period = 0
    v_neg_hold_win_period = 0
    for w, r, pre_pos, pos in zip(hold_win, hold_r, position.shift(1), position):
        """ 当日有换仓（先结算上一次持仓，再初始化本次持仓） """
        if pre_pos!=pos: 
            # 判断pre_pos非空：若为空则是循环的第一次，此时无需结算，直接初始化持仓即可
            if pre_pos == pre_pos:
                # 结算上一次持仓
                if pre_pos>0:
                    v_pos_hold_times += 1
                    v_pos_hold_period += tmp_hold_period
                    v_pos_hold_win_period += tmp_hold_win_period
                    if tmp_hold_r>0:
                        v_pos_hold_win_times+=1
                elif pre_pos<0:
                    v_neg_hold_times += 1      
                    v_neg_hold_period += tmp_hold_period
                    v_neg_hold_win_period += tmp_hold_win_period
                    if tmp_hold_r>0:                    
                        v_neg_hold_win_times+=1
            # 初始化本次持仓
            tmp_hold_r = r
            tmp_hold_period = 0
            tmp_hold_win_period = 0
        else: 
            if abs(pos)>0:
                tmp_hold_period += 1
                if r>0:
                    tmp_hold_win_period += 1
                if abs(r)>0:
                    tmp_hold_r = (1+tmp_hold_r)*(1+r)-1       

    v_hold_period = (abs(position)>0).sum()
    v_hold_win_period = (hold_r>0).sum()
    v_max_dd = drawdown.max()    
    
    v_annual_ret = (1+v_hold_cumu_r) ** (1/y_period) - 1
    v_annual_std = ex_hold_r.std() * np.sqrt( len(data)/y_period ) 
    v_sharpe= v_annual_ret / v_annual_std

    """ 生成Performance DataFrame """
    performance_cols = ['累计收益', 
                        '多仓次数', '多仓成功次数', '多仓胜率', '多仓平均持有期', 
                        '空仓次数', '空仓成功次数',  '空仓胜率', '空仓平均持有期', 
                        '周期胜率', '最大回撤', '年化收益/最大回撤',
                        '年化收益', '年化标准差', '年化夏普'
                       ]
    performance_values = ['{:.2%}'.format(v_hold_cumu_r),
                          v_pos_hold_times, v_pos_hold_win_times,
                                            '{:.2%}'.format(v_pos_hold_win_times/v_pos_hold_times if v_pos_hold_times != 0 else 0), 
                                            '{:.2f}'.format(v_pos_hold_period/v_pos_hold_times if v_pos_hold_times != 0 else 0),
                          v_neg_hold_times, v_neg_hold_win_times,
                                            '{:.2%}'.format(v_neg_hold_win_times / v_neg_hold_times if v_neg_hold_times != 0 else 0), 
                                            '{:.2f}'.format(v_neg_hold_win_times / v_neg_hold_times if v_neg_hold_times != 0 else 0),
                          '{:.2%}'.format(v_hold_win_period/v_hold_period if v_neg_hold_period != 0 else 0), 
                          '{:.2%}'.format(v_max_dd), 
                          '{:.2f}'.format(v_annual_ret/v_max_dd),
                          '{:.2%}'.format(v_annual_ret), 
                          '{:.2%}'.format(v_annual_std), 
                          '{:.2f}'.format(v_sharpe)
                         ]
    performance_df = pd.DataFrame(performance_values, index=performance_cols)
    
    return data, performance_df, v_hold_cumu_r/v_max_dd

## 测试遗传规划函数

In [44]:
def test(x):#遗传规划函数
    y= []
    for x1,x2,x3,x4,x5 in zip(x[:, 0],x[:,1],x[:, 2],x[:, 3],x[:, 4]):
        y.append(math.cos(x1*(math.cos(3*x2+3*x5)+3))+math.cos(x1-x2))
    return y   
"""
这些是跑出来的结果，效果都不好
"""        
#     math.cos(x5)+x4*math.sin(x2)/x3
#     2*protected_div(protected_acos(1),math.tan(x2))            
#     math.cos(x5)+math.cos(math.cos(protected_pow(x3,x2)*x1))
#     protected_div(x1, x2)+math.tan(-4).
#     x5 + math.sin(math.cos(x1)) - 4
#     math.cos(x1*(math.cos(3*x2+3*x5)+3))+math.cos(x1-x2)

    

'\n这些是跑出来的结果，效果都不好\n'

In [45]:
#训练集
my_y_train = np.sign(test(x_train))
my_data_train['gen_signal'] = my_y_train
my_signal_train = calc_signal(my_data_train, strategy=2)
my_position_train =calc_position(my_signal_train)
res_my_data_train, performance_df_train,fitnes_train = statistic_performance(my_position_train, 
                                                   data_period=1, 
                                                   is_consider_open=True,
                                                   point = 0.4,
                                                   is_daily_close=True,
                                                   comm=0.00005)

print(performance_df_train)
print(fitnes_train)

#测试集
my_y_test = np.sign(test(x_test))
my_data_test['gen_signal'] = my_y_test
my_signal_test = calc_signal(my_data_test, strategy=2)
my_position_test =calc_position(my_signal_test)
res_my_data_test, performance_df_test,fitnes_test = statistic_performance(my_position_test, 
                                                   data_period=1, 
                                                   is_consider_open=True,
                                                   point = 0.4,
                                                   is_daily_close=True,
                                                   comm=0.00005)
print(performance_df_test,fitnes_test)

                 0
累计收益       203.92%
多仓次数           386
多仓成功次数         152
多仓胜率        39.38%
多仓平均持有期      21.99
空仓次数           425
空仓成功次数         189
空仓胜率        44.47%
空仓平均持有期       0.44
周期胜率        50.43%
最大回撤         9.42%
年化收益/最大回撤     9.74
年化收益        91.80%
年化标准差       18.53%
年化夏普          4.95
21.637919834279618
                0
累计收益        5.26%
多仓次数          120
多仓成功次数         46
多仓胜率       38.33%
多仓平均持有期     23.61
空仓次数          169
空仓成功次数         73
空仓胜率       43.20%
空仓平均持有期      0.43
周期胜率       50.29%
最大回撤        7.47%
年化收益/最大回撤    1.12
年化收益        8.36%
年化标准差      14.98%
年化夏普         0.56 0.7048277025259511


In [46]:
#计算2010-2012年度收益表现
my_data_2010_2012 = my_data
x_2010_2012 = my_data_2010_2012[['high', 'low', 'close', 'volume', 'rank']].values
my_y_2010_2012 = np.sign(test(x_2010_2012))
my_data_2010_2012['gen_signal'] = my_y_2010_2012
my_signal_2010_2012 = calc_signal(my_data_2010_2012, strategy=2)
my_position_2010_2012 =calc_position(my_signal_2010_2012)
res_my_data_2010_2012, performance_df_2010_2012,fitnes_2010_2012 = statistic_performance(my_position_2010_2012, 
                                                   data_period=1, 
                                                   is_consider_open=True,
                                                   point = 0.4,                                      
                                                   is_daily_close=True,
                                                   comm=0.00005)

print(performance_df_2010_2012)
print(fitnes_2010_2012)

                 0
累计收益       221.87%
多仓次数           506
多仓成功次数         198
多仓胜率        39.13%
多仓平均持有期      22.38
空仓次数           595
空仓成功次数         263
空仓胜率        44.20%
空仓平均持有期       0.44
周期胜率        50.40%
最大回撤         9.42%
年化收益/最大回撤     6.81
年化收益        64.14%
年化标准差       17.59%
年化夏普          3.65
23.54175843167912


In [47]:
#分别计算2010 2011 2012年度收益表现
my_data_2010 = my_data[my_data['date']<='2011-01-01']
x_2010 = my_data_2010[['high', 'low', 'close', 'volume', 'rank']].values
my_y_2010 = np.sign(test(x_2010))
my_data_2010['gen_signal'] = my_y_2010
my_signal_2010 = calc_signal(my_data_2010, strategy=2)
my_position_2010 =calc_position(my_signal_2010)
res_my_data_2010, performance_df_2010,fitnes_2010 = statistic_performance(my_position_2010, 
                                                   data_period=1, 
                                                   is_consider_open=True,
                                                   point = 0.4,                       
                                                   is_daily_close=True,
                                                   comm=0.00005)

print(performance_df_2010)
print(fitnes_2010)

                 0
累计收益       113.37%
多仓次数           169
多仓成功次数          65
多仓胜率        38.46%
多仓平均持有期      20.75
空仓次数           190
空仓成功次数          86
空仓胜率        45.26%
空仓平均持有期       0.45
周期胜率        50.56%
最大回撤         7.14%
年化收益/最大回撤    26.73
年化收益       190.97%
年化标准差       21.95%
年化夏普          8.70
15.867976845595125


In [48]:
#分别计算2010 2011 2012年度收益表现
my_data_2011 = my_data[('2011-01-01'<=my_data['date'])& (my_data['date']<='2012-01-01')]
x_2011 = my_data_2011[['high', 'low', 'close', 'volume', 'rank']].values
my_y_2011 = np.sign(test(x_2011))
my_data_2011['gen_signal'] = my_y_2011
my_signal_2011 = calc_signal(my_data_2011, strategy=2)
my_position_2011 =calc_position(my_signal_2011)
res_my_data_2011, performance_df_2011,fitnes_2011 = statistic_performance(my_position_2011, 
                                                   data_period=1, 
                                                   is_consider_open=True,
                                                   point = 0.4,                       
                                                   is_daily_close=True,
                                                   comm=0.00005)

print(performance_df_2011)
print(fitnes_2011)

                0
累计收益       41.43%
多仓次数          216
多仓成功次数         86
多仓胜率       39.81%
多仓平均持有期     22.85
空仓次数          235
空仓成功次数        103
空仓胜率       43.83%
空仓平均持有期      0.44
周期胜率       50.33%
最大回撤        9.42%
年化收益/最大回撤    4.47
年化收益       42.11%
年化标准差      15.69%
年化夏普         2.68
4.395663250511434


In [49]:
#分别计算2010 2011 2012年度收益表现
my_data_2012 = my_data[('2012-01-01'<=my_data['date'])& (my_data['date']<='2013-01-01')]
x_2012 = my_data_2012[['high', 'low', 'close', 'volume', 'rank']].values
my_y_2012 = np.sign(test(x_2012))
my_data_2012['gen_signal'] = my_y_2012
my_signal_2012 = calc_signal(my_data_2012, strategy=2)
my_position_2012 =calc_position(my_signal_2012)
res_my_data_2012, performance_df_2012,fitnes_2012 = statistic_performance(my_position_2012, 
                                                   data_period=1, 
                                                   is_consider_open=True,
                                                   point = 0.4,                       
                                                   is_daily_close=True,
                                                   comm=0.00005)

print(performance_df_2012)
print(fitnes_2012)

                0
累计收益        5.26%
多仓次数          120
多仓成功次数         46
多仓胜率       38.33%
多仓平均持有期     23.61
空仓次数          169
空仓成功次数         73
空仓胜率       43.20%
空仓平均持有期      0.43
周期胜率       50.29%
最大回撤        7.47%
年化收益/最大回撤    1.12
年化收益        8.36%
年化标准差      14.98%
年化夏普         0.56
0.7048277025259511


In [50]:
""" 随机赋值 """#以下纯测试 无意义
np.random.seed(1)
my_data['gen_signal'] = np.sign(np.random.normal(size=len(my_data)))
# my_data.drop["dates"]#,"money"]

In [51]:
my_signal = calc_signal(my_data, strategy=2)


In [52]:
my_position =calc_position(my_signal)
# my_position[220:270]

In [53]:
res_my_data, performance_df,fitnes = statistic_performance(my_position, 
                                                   data_period=1, 
                                                   is_consider_open=True,
                                                   point = 0.4,        
                                                   is_daily_close = True,
                                                   comm=0.00005)

print(performance_df)
# res_my_data.query("hold_cumu_r<-0.1")
# res_my_data[220:270]
# res_my_data.to_csv('res_my_data.csv', index=False)

                0
累计收益       -4.14%
多仓次数          619
多仓成功次数        204
多仓胜率       32.96%
多仓平均持有期     20.90
空仓次数          590
空仓成功次数        231
空仓胜率       39.15%
空仓平均持有期      0.39
周期胜率       49.34%
最大回撤       29.42%
年化收益/最大回撤   -0.06
年化收益       -1.78%
年化标准差      17.55%
年化夏普        -0.10
